In [139]:
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F
from pyspark.sql.types import IntegerType, StringType


In [140]:
scSpark = SparkSession.builder.appName("Assignment5.3").getOrCreate()

In [141]:
# Read data from a source and create a DataFrame
colunms = ["PassengerId","Survived","class","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked","TimeStamp"]

titanic_df = scSpark.read.csv('./data/titanic.csv', sep=',', inferSchema=True, header=False)
titanic_df=titanic_df.toDF(*colunms)
titanic_df.show(5)

+-----------+--------+-----+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|class|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          TimeStamp|
+-----------+--------+-----+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|       0|    3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|2020-01-01 13:45:25|
|          2|       1|    1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|2020-01-01 13:44:48|
|          3|       1|    3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|2020-01-01 13:38:11|
|          4|       1|    1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|2020-01-01 13:32:00|
|          5|       0|    3|Allen, Mr. Willia...|  male| 35|    0|   

### Task 1
For numerical columns, calculate minimum, maximum and average values.


In [142]:
# Colunms
allcols = [cols[0] for cols in titanic_df]
allcols

# Separating colunms
cat_cols = [item[0] for item in titanic_df.dtypes if item[1].startswith('string')]
numeric_cols = [col for col in titanic_df.columns if col not in cat_cols]

cat_df = titanic_df[cat_cols]
numeric_df = titanic_df[numeric_cols[:-1]].drop('Survived')

# Summary
numeric_df.describe().show()

+-------+-----------------+------------------+------------------+------------------+-------------------+-----------------+
|summary|      PassengerId|             class|               Age|             SibSp|              Parch|             Fare|
+-------+-----------------+------------------+------------------+------------------+-------------------+-----------------+
|  count|              891|               891|               714|               891|                891|              891|
|   mean|            446.0| 2.308641975308642|29.679271708683473|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|257.3538420152301|0.8360712409770491|14.536482769437564|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                1|                 1|                 0|                 0|                  0|              0.0|
|    max|              891|                 3|                80|                 8|                  6|         512.3292|
+-------+-------

### Task 2 & 3
For categorical columns, create and apply UDF that will change the last letter of every word to “1”,

And Sort DataFrame by the first column and save the results to the Parquet file.




In [143]:
cols = ['Sex', 'Parch', 'class', 'Embarked']

def word_change(words): 
    words = str(words)
    if(len(words) == 1):
        return words[0:-1] + '1'
    return ' '.join([word[0:-1]+'1' for word in words.split()])

word_change_udf = F.udf(word_change, StringType())
for col in cols:
    titanic_df =  titanic_df.withColumn(col, word_change_udf(titanic_df[col]))
titanic_df.show(2)

final_df =  titanic_df.sort(titanic_df[0], asc=False)
final_df.write.mode("overwrite").save('./data/titanic.parquet')

+-----------+--------+-----+--------------------+------+---+-----+-----+---------+-------+-----+--------+-------------------+
|PassengerId|Survived|class|                Name|   Sex|Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|          TimeStamp|
+-----------+--------+-----+--------------------+------+---+-----+-----+---------+-------+-----+--------+-------------------+
|          1|       0|    1|Braund, Mr. Owen ...|  mal1| 22|    1|    1|A/5 21171|   7.25| null|       1|2020-01-01 13:45:25|
|          2|       1|    1|Cumings, Mrs. Joh...|femal1| 38|    1|    1| PC 17599|71.2833|  C85|       1|2020-01-01 13:44:48|
+-----------+--------+-----+--------------------+------+---+-----+-----+---------+-------+-----+--------+-------------------+
only showing top 2 rows

